# KEEN 新架構互動式分析測試
# Interactive Analysis Demo for KEEN New Architecture

這個 notebook 展示如何使用新的 KEEN 架構進行互動式 SPM 數據分析。

This notebook demonstrates how to use the new KEEN architecture for interactive SPM data analysis.

## 1. 環境設定 / Environment Setup

In [ ]:
# 添加後端路徑到 Python 路徑
import sys
from pathlib import Path
backend_path = Path().resolve().parent
sys.path.insert(0, str(backend_path))

# 導入必要的模組
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import logging

# 導入 KEEN 新架構
from core.experiment_session import ExperimentSession
from core.data_models import TopoData, CitsData, StsData, TxtData

# 設定日誌
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 環境設定完成 / Environment setup complete")

## 2. Plotly 繪圖工具函數 / Plotly Plotting Utility Functions

In [ ]:
def plot_topography_plotly(topo_data: TopoData, title: str = "Topography"):
    """
    使用 Plotly 繪製拓撲圖
    Plot topography using Plotly
    """
    image = topo_data.current_image
    
    # 創建座標軸
    x_coords = np.linspace(0, topo_data.x_range, image.shape[1])
    y_coords = np.linspace(0, topo_data.y_range, image.shape[0])
    
    fig = go.Figure(data=go.Heatmap(
        z=image,
        x=x_coords,
        y=y_coords,
        colorscale='Viridis',
        colorbar=dict(title="Height (pm)")
    ))
    
    fig.update_layout(
        title=f"{title} - {topo_data.signal_type} {topo_data.direction or ''}",
        xaxis_title="X Position (nm)",
        yaxis_title="Y Position (nm)",
        width=600,
        height=500,
        yaxis=dict(scaleanchor="x", scaleratio=1)  # 保持縱橫比
    )
    
    return fig

def plot_line_profile_plotly(profile_data: dict, title: str = "Line Profile"):
    """
    使用 Plotly 繪製線段剖面
    Plot line profile using Plotly
    """
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=profile_data['distance'],
        y=profile_data['height'],
        mode='lines+markers',
        name='Height Profile',
        line=dict(width=2)
    ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Distance (nm)",
        yaxis_title="Height (pm)",
        width=700,
        height=400
    )
    
    return fig

def plot_cits_overview_plotly(cits_data: CitsData, title: str = "CITS Overview"):
    """
    使用 Plotly 繪製 CITS 概覽
    Plot CITS overview using Plotly
    """
    # 選擇中間偏壓的切片
    mid_bias_idx = len(cits_data.bias_values) // 2
    slice_data = cits_data.get_bias_slice(mid_bias_idx)
    bias_value = cits_data.bias_values[mid_bias_idx]
    
    fig = go.Figure(data=go.Heatmap(
        z=slice_data,
        colorscale='RdBu',
        colorbar=dict(title="Current (pA)")
    ))
    
    fig.update_layout(
        title=f"{title} - Bias: {bias_value:.3f} V",
        xaxis_title="X Pixel",
        yaxis_title="Y Pixel",
        width=600,
        height=500
    )
    
    return fig

def plot_sts_spectra_plotly(sts_data: StsData, title: str = "STS Spectra"):
    """
    使用 Plotly 繪製 STS 光譜
    Plot STS spectra using Plotly
    """
    fig = go.Figure()
    
    # 繪製平均光譜
    avg_spectrum = np.mean(sts_data.data_2d, axis=1)
    fig.add_trace(go.Scatter(
        x=sts_data.bias_values,
        y=avg_spectrum,
        mode='lines',
        name='Average Spectrum',
        line=dict(width=3, color='red')
    ))
    
    # 繪製前幾條個別光譜（半透明）
    n_show = min(5, sts_data.n_points)
    for i in range(n_show):
        fig.add_trace(go.Scatter(
            x=sts_data.bias_values,
            y=sts_data.data_2d[:, i],
            mode='lines',
            name=f'Point {i+1}',
            line=dict(width=1),
            opacity=0.5
        ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Bias Voltage (V)",
        yaxis_title="Current (pA)",
        width=700,
        height=500
    )
    
    return fig

print("✅ Plotly 繪圖函數載入完成 / Plotly plotting functions loaded")

## 3. 檔案載入 / File Loading

In [ ]:
# 設定測試檔案目錄
testfile_dir = backend_path.parent / "testfile"
print(f"📁 測試檔案目錄: {testfile_dir}")

# 尋找可用的 TXT 檔案
txt_files = list(testfile_dir.glob("*.txt"))
print(f"🔍 找到 {len(txt_files)} 個 TXT 檔案:")
for i, txt_file in enumerate(txt_files):
    print(f"  {i+1}. {txt_file.name}")

if not txt_files:
    print("❌ 未找到 TXT 檔案，請確保 testfile 目錄中有實驗檔案")
else:
    # 使用第一個檔案
    selected_txt = txt_files[0]
    print(f"\n📄 載入檔案: {selected_txt.name}")
    
    try:
        # 初始化實驗會話
        session = ExperimentSession(str(selected_txt))
        print(f"✅ 實驗會話建立成功: {session.experiment_name}")
        
        # 顯示會話摘要
        summary = session.get_session_summary()
        print(f"\n📊 實驗摘要:")
        print(f"  總檔案數: {summary['files_summary']['total_available']}")
        print(f"  TXT: {summary['files_summary']['available']['txt']}")
        print(f"  TOPO: {summary['files_summary']['available']['topo']}")
        print(f"  CITS: {summary['files_summary']['available']['cits']}")
        print(f"  STS: {summary['files_summary']['available']['sts']}")
        
    except Exception as e:
        print(f"❌ 載入失敗: {str(e)}")
        session = None

## 4. 互動式檔案選擇器 / Interactive File Selector

In [ ]:
if session:
    # 創建檔案選擇 widgets
    available_files = session.available_files
    
    # TOPO 檔案選擇器
    topo_files = available_files['topo']
    topo_dropdown = widgets.Dropdown(
        options=[(f"{f} (TOPO)", f) for f in topo_files],
        description='TOPO檔案:',
        disabled=len(topo_files) == 0,
        style={'description_width': 'initial'}
    )
    
    # CITS 檔案選擇器
    cits_files = available_files['cits']
    cits_dropdown = widgets.Dropdown(
        options=[(f"{f} (CITS)", f) for f in cits_files],
        description='CITS檔案:',
        disabled=len(cits_files) == 0,
        style={'description_width': 'initial'}
    )
    
    # STS 檔案選擇器
    sts_files = available_files['sts']
    sts_dropdown = widgets.Dropdown(
        options=[(f"{f} (STS)", f) for f in sts_files],
        description='STS檔案:',
        disabled=len(sts_files) == 0,
        style={'description_width': 'initial'}
    )
    
    # 分析選項
    analysis_options = widgets.SelectMultiple(
        options=[
            ('基本分析 / Basic Analysis', 'basic'),
            ('平坦化 / Flattening', 'flatten'),
            ('線段剖面 / Line Profile', 'profile'),
            ('峰值檢測 / Peak Detection', 'peaks'),
            ('能隙分析 / Gap Analysis', 'gap')
        ],
        value=['basic'],
        description='分析類型:',
        style={'description_width': 'initial'}
    )
    
    # 分析按鈕
    analyze_button = widgets.Button(
        description='🔬 開始分析 / Start Analysis',
        button_style='primary',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # 輸出區域
    output_area = widgets.Output()
    
    # 顯示 widgets
    display(HTML("<h3>📋 選擇要分析的檔案和選項 / Select Files and Options for Analysis</h3>"))
    display(widgets.VBox([
        topo_dropdown,
        cits_dropdown, 
        sts_dropdown,
        analysis_options,
        analyze_button
    ]))
    
    print("✅ 互動式介面建立完成 / Interactive interface created")
else:
    print("❌ 無法建立互動式介面，請先載入實驗檔案")

## 5. 分析函數定義 / Analysis Function Definitions

In [ ]:
def analyze_topo_file(session, file_key, analysis_types):
    """
    分析拓撲檔案
    Analyze topography file
    """
    try:
        print(f"🔬 分析拓撲檔案: {file_key}")
        topo = session[file_key]
        
        results = {}
        
        # 基本分析
        if 'basic' in analysis_types:
            print("  📊 執行基本分析...")
            analyzer = topo.analyzer
            basic_result = analyzer.analyze()
            results['basic'] = basic_result
            
            # 顯示基本資訊
            if basic_result['success']:
                topo_info = basic_result['data']['topo_info']
                print(f"    圖像尺寸: {topo_info['shape']}")
                print(f"    數據範圍: {topo_info['data_range'][0]:.2f} ~ {topo_info['data_range'][1]:.2f} pm")
                print(f"    像素尺度: {topo_info['pixel_scale'][0]:.3f} x {topo_info['pixel_scale'][1]:.3f} nm/pixel")
                
                # 繪製拓撲圖
                fig = plot_topography_plotly(topo.data, "Original Topography")
                fig.show()
        
        # 平坦化分析
        if 'flatten' in analysis_types:
            print("  🏔️ 執行平坦化...")
            analyzer = topo.analyzer
            flatten_result = analyzer.apply_flattening('linewise_mean')
            results['flatten'] = flatten_result
            
            if flatten_result['success']:
                print("    ✅ 平坦化完成")
                # 繪製平坦化後的圖像
                fig = plot_topography_plotly(topo.data, "Flattened Topography")
                fig.show()
        
        # 線段剖面
        if 'profile' in analysis_types:
            print("  📏 提取線段剖面...")
            analyzer = topo.analyzer
            
            # 使用圖像中心的水平線
            height, width = topo.data.shape
            start_point = (height//2, 0)
            end_point = (height//2, width-1)
            
            profile_result = analyzer.extract_line_profile(start_point, end_point)
            results['profile'] = profile_result
            
            if profile_result['success']:
                print("    ✅ 線段剖面提取完成")
                # 繪製線段剖面
                fig = plot_line_profile_plotly(profile_result['data'], "Height Profile")
                fig.show()
        
        return results
        
    except Exception as e:
        print(f"    ❌ 拓撲分析失敗: {str(e)}")
        return {'error': str(e)}

def analyze_cits_file(session, file_key, analysis_types):
    """
    分析 CITS 檔案
    Analyze CITS file
    """
    try:
        print(f"🔬 分析 CITS 檔案: {file_key}")
        cits = session[file_key]
        
        results = {}
        
        # 基本分析
        if 'basic' in analysis_types:
            print("  📊 執行基本分析...")
            analyzer = cits.analyzer
            basic_result = analyzer.analyze()
            results['basic'] = basic_result
            
            if basic_result['success']:
                cits_info = basic_result['data']['cits_data_info']
                print(f"    數據形狀: {cits_info['shape']}")
                print(f"    偏壓點數: {cits_info['n_bias_points']}")
                print(f"    偏壓範圍: {cits_info['bias_range'][0]:.3f} ~ {cits_info['bias_range'][1]:.3f} V")
                
                # 繪製 CITS 概覽
                fig = plot_cits_overview_plotly(cits.data, "CITS Overview")
                fig.show()
        
        return results
        
    except Exception as e:
        print(f"    ❌ CITS 分析失敗: {str(e)}")
        return {'error': str(e)}

def analyze_sts_file(session, file_key, analysis_types):
    """
    分析 STS 檔案
    Analyze STS file
    """
    try:
        print(f"🔬 分析 STS 檔案: {file_key}")
        sts = session[file_key]
        
        results = {}
        
        # 基本分析
        if 'basic' in analysis_types:
            print("  📊 執行基本分析...")
            analyzer = sts.analyzer
            basic_result = analyzer.analyze()
            results['basic'] = basic_result
            
            if basic_result['success']:
                sts_info = basic_result['data']['sts_data_info']
                print(f"    數據形狀: {sts_info['shape']}")
                print(f"    測量點數: {sts_info['n_points']}")
                print(f"    偏壓點數: {sts_info['n_bias_points']}")
                
                # 繪製 STS 光譜
                fig = plot_sts_spectra_plotly(sts.data, "STS Spectra")
                fig.show()
        
        # 峰值檢測
        if 'peaks' in analysis_types:
            print("  🏔️ 執行峰值檢測...")
            analyzer = sts.analyzer
            peak_result = analyzer.detect_peaks(threshold_ratio=0.1)
            results['peaks'] = peak_result
            
            if peak_result['success']:
                peaks = peak_result['data']['peaks']
                print(f"    找到 {peaks['n_peaks']} 個峰值")
                if peaks['n_peaks'] > 0:
                    print(f"    峰值位置: {[f'{bias:.3f}V' for bias in peaks['peak_biases']]}")
        
        # 能隙分析
        if 'gap' in analysis_types:
            print("  🕳️ 執行能隙分析...")
            analyzer = sts.analyzer
            gap_result = analyzer.analyze_gap(gap_method='minimum')
            results['gap'] = gap_result
            
            if gap_result['success']:
                gap_info = gap_result['data']['gap_info']
                print(f"    能隙中心: {gap_info['gap_center']:.3f} V")
                print(f"    能隙寬度: {gap_info['gap_width']:.3f} V")
        
        return results
        
    except Exception as e:
        print(f"    ❌ STS 分析失敗: {str(e)}")
        return {'error': str(e)}

print("✅ 分析函數定義完成 / Analysis functions defined")

## 6. 按鈕點擊事件處理 / Button Click Event Handler

In [ ]:
def on_analyze_button_clicked(b):
    """
    分析按鈕點擊事件處理
    Handle analyze button click event
    """
    with output_area:
        clear_output(wait=True)
        
        print("🚀 開始分析... / Starting analysis...")
        print("=" * 60)
        
        analysis_types = list(analysis_options.value)
        all_results = {}
        
        # 分析 TOPO 檔案
        if not topo_dropdown.disabled and topo_dropdown.value:
            print("\n🖼️ 拓撲分析 / Topography Analysis")
            print("-" * 40)
            topo_results = analyze_topo_file(session, topo_dropdown.value, analysis_types)
            all_results['topo'] = topo_results
        
        # 分析 CITS 檔案
        if not cits_dropdown.disabled and cits_dropdown.value:
            print("\n🔬 CITS 分析 / CITS Analysis")
            print("-" * 40)
            cits_results = analyze_cits_file(session, cits_dropdown.value, analysis_types)
            all_results['cits'] = cits_results
        
        # 分析 STS 檔案
        if not sts_dropdown.disabled and sts_dropdown.value:
            print("\n📊 STS 分析 / STS Analysis")
            print("-" * 40)
            sts_results = analyze_sts_file(session, sts_dropdown.value, analysis_types)
            all_results['sts'] = sts_results
        
        print("\n" + "=" * 60)
        print("✅ 分析完成！/ Analysis Complete!")
        
        # 顯示總結
        print("\n📋 分析總結 / Analysis Summary:")
        for file_type, results in all_results.items():
            if 'error' not in results:
                successful_analyses = [k for k, v in results.items() if v.get('success', False)]
                print(f"  {file_type.upper()}: {len(successful_analyses)} 項分析成功")
            else:
                print(f"  {file_type.upper()}: 分析失敗")

# 綁定按鈕事件
if session:
    analyze_button.on_click(on_analyze_button_clicked)
    
    # 顯示輸出區域
    display(HTML("<h3>📊 分析結果 / Analysis Results</h3>"))
    display(output_area)
    
    print("✅ 事件處理器設定完成 / Event handlers configured")
else:
    print("❌ 無法設定事件處理器，請先載入實驗檔案")

## 7. 進階分析功能 / Advanced Analysis Features

In [ ]:
if session:
    # 創建進階分析 widgets
    print("🔧 進階分析工具 / Advanced Analysis Tools")
    print("=" * 50)
    
    # 如果有 CITS 檔案，提供偏壓切片功能
    if available_files['cits']:
        print("\n📐 CITS 偏壓切片分析 / CITS Bias Slice Analysis")
        
        # 獲取第一個 CITS 檔案的偏壓範圍
        first_cits = session[available_files['cits'][0]]
        cits_data = first_cits.data
        
        bias_slider = widgets.IntSlider(
            value=len(cits_data.bias_values) // 2,
            min=0,
            max=len(cits_data.bias_values) - 1,
            step=1,
            description='偏壓索引:',
            style={'description_width': 'initial'}
        )
        
        bias_value_label = widgets.Label(
            value=f"偏壓值: {cits_data.bias_values[bias_slider.value]:.3f} V"
        )
        
        def update_bias_label(change):
            bias_value_label.value = f"偏壓值: {cits_data.bias_values[change['new']]:.3f} V"
        
        bias_slider.observe(update_bias_label, names='value')
        
        slice_button = widgets.Button(
            description='📊 顯示偏壓切片',
            button_style='info'
        )
        
        slice_output = widgets.Output()
        
        def show_bias_slice(b):
            with slice_output:
                clear_output(wait=True)
                try:
                    bias_idx = bias_slider.value
                    slice_data = cits_data.get_bias_slice(bias_idx)
                    bias_value = cits_data.bias_values[bias_idx]
                    
                    fig = go.Figure(data=go.Heatmap(
                        z=slice_data,
                        colorscale='RdBu',
                        colorbar=dict(title="Current (pA)")
                    ))
                    
                    fig.update_layout(
                        title=f"CITS 偏壓切片 - {bias_value:.3f} V (索引 {bias_idx})",
                        xaxis_title="X Pixel",
                        yaxis_title="Y Pixel",
                        width=600,
                        height=500
                    )
                    
                    fig.show()
                    print(f"✅ 顯示偏壓切片: {bias_value:.3f} V")
                    
                except Exception as e:
                    print(f"❌ 顯示偏壓切片失敗: {str(e)}")
        
        slice_button.on_click(show_bias_slice)
        
        display(widgets.VBox([
            bias_slider,
            bias_value_label,
            slice_button
        ]))
        display(slice_output)
    
    print("\n✅ 進階分析工具設定完成 / Advanced analysis tools configured")
else:
    print("❌ 無法設定進階分析工具，請先載入實驗檔案")

## 8. 會話資訊顯示 / Session Information Display

In [ ]:
if session:
    print("📊 實驗會話詳細資訊 / Detailed Session Information")
    print("=" * 60)
    
    # 獲取會話摘要
    summary = session.get_session_summary()
    
    print(f"📁 實驗名稱: {summary['experiment_name']}")
    print(f"📅 建立時間: {summary['creation_time']}")
    print(f"📄 TXT 檔案: {summary['txt_file']}")
    
    # 掃描參數
    if summary['scan_parameters']:
        scan_params = summary['scan_parameters']
        print(f"\n🔬 掃描參數:")
        print(f"  解析度: {scan_params['x_pixel']} × {scan_params['y_pixel']} pixels")
        print(f"  掃描範圍: {scan_params['x_range']} × {scan_params['y_range']} nm")
        print(f"  像素尺度: {scan_params['pixel_scale_x']:.3f} × {scan_params['pixel_scale_y']:.3f} nm/pixel")
        print(f"  長寬比: {scan_params['aspect_ratio']:.3f}")
        print(f"  總像素數: {scan_params['total_pixels']:,}")
    
    # 檔案統計
    files_summary = summary['files_summary']
    print(f"\n📋 檔案統計:")
    print(f"  可用檔案: {files_summary['total_available']}")
    print(f"  已載入檔案: {files_summary['total_loaded']}")
    print(f"  詳細分布:")
    for file_type, count in files_summary['available'].items():
        loaded_count = files_summary['loaded'][file_type]
        print(f"    {file_type.upper()}: {count} 可用, {loaded_count} 已載入")
    
    # 記憶體使用
    memory_info = summary['memory_info']
    print(f"\n💾 記憶體使用:")
    print(f"  代理快取大小: {memory_info['proxy_cache_size']}")
    print(f"  各類型快取:")
    for cache_type, cache_info in memory_info.items():
        if isinstance(cache_info, dict) and 'cache_size' in cache_info:
            hit_rate = cache_info.get('hit_rate', 0) * 100
            print(f"    {cache_type}: {cache_info['cache_size']} 項目, 命中率: {hit_rate:.1f}%")
    
    print("\n✅ 會話資訊顯示完成 / Session information display complete")
else:
    print("❌ 無會話資訊可顯示")

## 9. 測試總結與說明 / Test Summary and Instructions

### 🎉 新架構測試完成！

這個 notebook 展示了 KEEN 新架構的主要功能：

1. **📁 統一的檔案管理** - `ExperimentSession` 自動載入和管理所有相關檔案
2. **🔧 直覺的數據存取** - `session['file'].data.attribute` 語法，完整 IDE 支援
3. **🧠 智能分析器** - 每種數據類型都有專門的分析器
4. **📊 互動式分析** - Jupyter widgets 提供友好的使用者介面
5. **📈 Plotly 整合** - 高品質的互動式圖表

### 🚀 使用說明：

1. 執行所有儲存格來設定環境
2. 選擇要分析的檔案類型和分析選項
3. 點擊「🔬 開始分析」按鈕
4. 查看自動生成的 Plotly 圖表和分析結果
5. 使用進階工具（如 CITS 偏壓切片）進行深入分析

### ✨ 新架構優勢：

- ✅ **型別安全** - 完整的 TypeScript 風格型別提示
- ✅ **狀態持久** - 分析結果自動儲存在對應的數據物件中
- ✅ **模組化設計** - 清晰的職責分離
- ✅ **向後兼容** - 現有的分析算法無縫整合
- ✅ **易於擴展** - 新的分析功能可輕鬆添加
